# Section 3 - WebScraping : EU Construction Industry

webscraping:https://towardsdatascience.com/web-scraping-news-articles-in-python-9dd605799558

## Section 3.1 Importing Libraries

In [59]:
#importing required libraries
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC

In [82]:
#https://www.youtube.com/watch?v=71hlQDBbbLM&list=PLz-0BiySzeQUuXoD4_mT3lex6HNi7f-YH&index=2
# webdriver_service = Service('C:\webdrivers\chromedriver.exe')
# driver = webdriver.Chrome(options=options, service=webdriver_service)
# driver.get(url)

# wait = WebDriverWait(driver, 15)
# element=driver.find_element_by_xpath('//*[@id="didomi-notice-agree-button"]')
# element.click()
# time.sleep(2)
# #driver.quit()

## Section 3.2 Webscraping from Independent

In [18]:
#defining the URl - query search on independent for topics realted to construction industry and housing in Ireland
url = "https://www.independent.ie/search?keyword=construction+industry+housing+ireland&daterange=all&datestart=&dateend="

In [19]:
#pulling the request
r1 = requests.get(url)
r1.status_code

#creating soup, using the html parser
soup1 = BeautifulSoup(coverpage, 'html.parser')

#identifying the news items from the HTML CSS
headlines_news = soup1.find_all('li', class_='indo-d382f4d9_marginbottom5 indo-ddd60156_paddingbottom5 indo-ddd60156_paddingleft0 indo-ddd60156_paddingright0 indo-ddd60156_paddingtop0 indo-9d6383c7_listitem indo-9d6383c7_listitemdivider')  #vhange this
len(headlines_news)

10

In [83]:
number_of_articles = len(headlines_news)

In [84]:
#creating empty lists
news_contents = []
list_links = []
list_titles = []

for n in np.arange(0, number_of_articles):
            
    #retreiving the link for the article
    link = headlines_news[n].find('a')['href']
    list_links.append(link)
    
    #retriving the title of the article
    title = headlines_news[n].find('a').get_text()
    list_titles.append(title)
    
    #reading in the content that can be accessed without subscription
    article = requests.get(f'https://www.independent.ie{link}') #adding the inital link to the article link
    article_content = article.content
    soup_article = BeautifulSoup(article_content, 'html.parser')
    body = soup_article.find_all('div', class_='indo-1d70522a_marginbottom5 indo-1d70522a_marginleft0 indo-1d70522a_marginright0 indo-26838ee5_paddingleft0 indo-26838ee5_paddingright0 indo-7199e23_root indo-804a7bc3_root')
    x = body[0].find_all('p')
    
    #getting the text from the articles
    list_paragraphs = []
    for p in np.arange(0, len(x)):
        paragraph = x[p].get_text()
        list_paragraphs.append(paragraph)
        final_article = " ".join(list_paragraphs)
        
    news_contents.append(final_article)

In [85]:
#creating dataframes from the list data and merging into data frames
list_titles_df=pd.DataFrame(list_titles, columns=['Title'])
news_contents_df=pd.DataFrame(news_contents, columns=['Content'])
final_data=pd.merge(list_titles_df, news_contents_df, how='outer', left_index=True, right_index=True)
final_data

,Title,Content
0,Homes Home Truths: Why a third of all new home...,What if a full one third of all the new homes ...
1,Irish Business Tom McEnaney: Builders of the w...,The critical issue impacting the number of hou...
2,Comment Eoin O’Malley: Ireland needs more buil...,The Government was excoriated in the Dáil last...
3,Business Public Private Partnership’s bad rap ...,Ray Wilson knows public private partnership (P...
4,Irish News Housing Minister Darragh O’Brien on...,"If self-confidence built homes, Darragh O’Brie..."
5,Personal Finance One solution to the housing c...,Declan Dunne rarely focuses on blame or on the...
6,Irish Business More than half of developers ha...,Over half of housing developers have postponed...
7,Personal Finance Richard Curran: I warned of a...,They say the past is a different country. And ...
8,World Business Spectre of 2008 crisis hovers o...,The collapse of Silicon Valley Bank (SVB) and ...
9,"Irish News An extra 50,000 homes could be deli...","An extra 50,000 homes can be delivered in the ..."


In [88]:
#exporting to a csv
final_data.to_csv('webscraped_data.csv', index=False)